<a href="https://colab.research.google.com/github/NeelayS/speech_spike_signatures/blob/master/PyGeNN_MNIST_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!git clone https://github.com/genn-team/genn.git --branch 4.1.0

Cloning into 'genn'...
remote: Enumerating objects: 3116, done.
remote: Counting objects: 100% (3116/3116), done.
remote: Compressing objects: 100% (1611/1611), done.
remote: Total 79777 (delta 2004), reused 2220 (delta 1436), pack-reused 76661
Receiving objects: 100% (79777/79777), 223.71 MiB | 14.72 MiB/s, done.
Resolving deltas: 100% (55769/55769), done.
Note: checking out '37e7a83b1c2c5cfa158e46f4290acb24a58d6e7f'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



In [35]:
!sudo apt-get install swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [36]:
import os
os.environ['CUDA_PATH'] ='/usr/local/cuda'
os.chdir('genn/')
!make DYNAMIC=1 LIBRARY_DIRECTORY=`pwd`/pygenn/genn_wrapper/

make -C src/genn/genn
make[1]: Entering directory '/content/genn/genn/src/genn/genn'
mkdir -p /content/genn/genn/obj_dynamic/genn/genn
g++ -std=c++11 -Wall -Wpedantic -Wextra -MMD -MP -I/content/genn/genn/include/genn/genn -I/content/genn/genn/include/genn/third_party -fPIC -c -o /content/genn/genn/obj_dynamic/genn/genn/synapseGroup.o synapseGroup.cc
mkdir -p /content/genn/genn/obj_dynamic/genn/genn
g++ -std=c++11 -Wall -Wpedantic -Wextra -MMD -MP -I/content/genn/genn/include/genn/genn -I/content/genn/genn/include/genn/third_party -fPIC -c -o /content/genn/genn/obj_dynamic/genn/genn/neuronModels.o neuronModels.cc
mkdir -p /content/genn/genn/obj_dynamic/genn/genn
g++ -std=c++11 -Wall -Wpedantic -Wextra -MMD -MP -I/content/genn/genn/include/genn/genn -I/content/genn/genn/include/genn/third_party -fPIC -c -o /content/genn/genn/obj_dynamic/genn/genn/initVarSnippet.o initVarSnippet.cc
mkdir -p /content/genn/genn/obj_dynamic/genn/genn
g++ -std=c++11 -Wall -Wpedantic -Wextra -MMD -MP -I/conte

In [37]:
!python setup.py develop

running develop
running egg_info
creating pygenn.egg-info
writing pygenn.egg-info/PKG-INFO
writing dependency_links to pygenn.egg-info/dependency_links.txt
writing requirements to pygenn.egg-info/requires.txt
writing top-level names to pygenn.egg-info/top_level.txt
writing manifest file 'pygenn.egg-info/SOURCES.txt'
writing manifest file 'pygenn.egg-info/SOURCES.txt'
running build_ext
building '_StlContainers' extension
swigging pygenn/genn_wrapper/generated/StlContainers.i to pygenn/genn_wrapper/generated/StlContainers_wrap.cpp
swig -python -c++ -relativeimport -outdir /content/genn/genn/pygenn/genn_wrapper -I/content/genn/genn/pygenn/genn_wrapper/include -I/content/genn/genn/pygenn/genn_wrapper/generated -I/content/genn/genn/pygenn/genn_wrapper/swig -py3 -o pygenn/genn_wrapper/generated/StlContainers_wrap.cpp pygenn/genn_wrapper/generated/StlContainers.i
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/pygenn
creating build/temp.linux-x86_64-3.

In [38]:
!git clone https://github.com/neworderofjamie/pygenn_ml_tutorial.git

Cloning into 'pygenn_ml_tutorial'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 34 (delta 17), reused 27 (delta 10), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [39]:
import numpy as np
from os import path

In [40]:
from pygenn.genn_model import (create_custom_neuron_class, 
                               create_custom_current_source_class,
                               GeNNModel)
from pygenn.genn_wrapper import NO_DELAY

In [41]:
# Parameters

IF_PARAMS = {"Vthr": 5.0}           # Spiking threshold
TIMESTEP = 1.0
PRESENT_TIMESTEPS = 100             # No. of timesteps one sample is presented
INPUT_CURRENT_SCALE = 1.0/100.0     # Scaling image intensity to input currents

In [42]:
# Custom GeNN model

# Integrate-and-fire

# Blueprint of all neurons in the network

# Parameters are common for all neurons in the network (here Vthr)

if_model = create_custom_neuron_class(
    "if_model",                                                             # name of model
    param_names=["Vthr"],                 
    var_name_types=[("V", "scalar"), ("SpikeCount", "unsigned int")],       # State variables that track every neuron's membrane voltage and spike count
    sim_code="$(V) += $(Isyn) * DT;",                                       # Specifies that neurons simply integrate input current every timestep
    reset_code="""
    $(V) = 0.0;                                                             
    $(SpikeCount)++;
    """,
    threshold_condition_code="$(V) >= $(Vthr)")                             # Specifies that neurons spike when their voltage goes above a threshold Vth

# reset_code -> specifies that voltage is reset to 0 and spike count is incremented for a neuron whenever it fires

In [43]:
# Current source model 

# Injects current specified by the state variables every timestep

cs_model = create_custom_current_source_class(
    "cs_model",                                                              # name of model
    var_name_types=[("magnitude", "scalar")],                                # state variables
    injection_code="$(injectCurrent, $(magnitude));")                        # injection_code -> string with the current injection code

In [44]:
# Building the network

# Create GeNN model

model = GeNNModel("float", "tutorial_1")                                    # tutorial_1 is the directory name where the simulation code is generated
model.dT = TIMESTEP                                                         # simulation timestep

In [45]:
!ls

bin			     Jenkinsfile  py_filter.sh	      src
build			     lib	  pygenn	      tests
codecov.yml		     LICENSE.txt  pygenn.egg-info     userproject
CODE_OF_CONDUCT.md	     makedoc.sh   pygenn_ml_tutorial  version.txt
doxygen			     Makefile	  README.md	      www
generate_swig_interfaces.py  obj_dynamic  README.txt
genn.sln		     publishdoc   setup.py
include			     __pycache__  spineml.sln


In [46]:
# Load weights if already present into a list

weights = []
while True:
    filename = "pygenn_ml_tutorial/weights_%u_%u.npy" % (len(weights), len(weights) + 1)
    if path.exists(filename):
        weights.append(np.load(filename))
    else:
        break

In [47]:
type(weights)

list

In [48]:
len(weights)

2

In [49]:
weights[0].shape, weights[1].shape

((784, 128), (128, 10))

In [50]:
# Initial values for all IF neurons
if_init = {"V": 0.0, "SpikeCount":0}

In [51]:
# Create first neuron layer                                                             # http://genn-team.github.io/genn/documentation/4/html/db/d57/classpygenn_1_1genn__model_1_1GeNNModel.html#a100324e214546094fced5e66fb0582d4
neuron_layers = [model.add_neuron_population("neuron0", weights[0].shape[0],            # 1 neuron for each of the first layer's input dimensions
                                             if_model, IF_PARAMS, if_init)]             # population -> layer

# Create subsequent neuron layer
for i, w in enumerate(weights):
    neuron_layers.append(model.add_neuron_population("neuron%u" % (i + 1),
                                                     w.shape[1], if_model,
                                                     IF_PARAMS, if_init))

In [52]:
# Create synaptic connections between layers
for i, (pre, post, w) in enumerate(zip(neuron_layers[:-1], neuron_layers[1:], weights)):
    model.add_synapse_population(
        "synapse%u" % i, "DENSE_INDIVIDUALG", NO_DELAY,
        pre, post,                                              # pre -> source population, post -> target population
        "StaticPulse", {}, {"g": w.flatten()}, {}, {},          # StaticPulse -> weight update model (static here because it is used with pretrained weights)
        "DeltaCurr", {}, {})

#  add_synapse_population docs - http://genn-team.github.io/genn/documentation/4/html/db/d57/classpygenn_1_1genn__model_1_1GeNNModel.html#ab573f92c82ec25d5c80f94070b6008c0  

#  "synapse%u" % i -> name of synapse population

#  "DENSE_INDIVIDUALG" -> dense matrix with individual state varibales(weights) for each synapse  (http://genn-team.github.io/genn/documentation/4/html/dd/dd5/synapseMatrixType_8h.html#a24a045033b9a7e987843a67ff5ddec9c)

#  NO_DELAY -> no synaptic delays (can specify delay in terms of no. of simulation timesteps)   (http://genn-team.github.io/genn/documentation/4/html/db/d57/classpygenn_1_1genn__model_1_1GeNNModel.html#ab573f92c82ec25d5c80f94070b6008c0)

#  StaticPulse -> weight update model (static here because it is used with pretrained weights)

#  {} -> parameters for synapse updates (learning rule ?)

#  {"g": w.flatten()} -> initial values of synapse weights. Here, Initialise the weight of each synapse to the pre-trained weights

#  {} -> dict with initial values for WeightUpdateModels presynaptic variables

#  {} -> 	dict with initial values for WeightUpdateModels postsynaptic variables

#  DeltaCurr -> built-in delta postsynaptic model     (?)

#  {} -> dict with param values for the PostsynapticModels class
                                                                           # built-in delta postsynaptic model has no parameters or variables
#  {} -> dict with initial variable values for the PostsynapticModels class 

In [53]:
# Create current source to deliver input to first layers of neurons
current_input = model.add_current_source("current_input", cs_model,
                                         "neuron0" , {}, {"magnitude": 0.0})

# cs_model -> current source model (defined earlier)
# neuron0 -> population into which the current source should be injected
# {} -> dict with param values for cs_model
# {"magnitude": 0.0} -> 	dict with initial variable values for cs_model

In [54]:
# Build and load our model
model.build()
model.load()

## Simulation

In [55]:
# Load MNIST testing data
testing_images = np.load("pygenn_ml_tutorial/testing_images.npy")
testing_labels = np.load("pygenn_ml_tutorial/testing_labels.npy")


In [56]:
testing_images.shape, testing_labels.shape

((10000, 784), (10000,))

In [57]:
weights[0].shape, weights[1].shape

((784, 128), (128, 10))

In [58]:
# Check dimensions match network
assert testing_images.shape[1] == weights[0].shape[0]
assert np.max(testing_labels) == (weights[1].shape[1] - 1)

In [61]:
# Set current input by scaling first image
current_input.vars["magnitude"].view[:] = testing_images[0] * INPUT_CURRENT_SCALE

# Copy first image into memory view of current source magnitude

In [ ]:
# Upload
current_input.push_var_to_device("magnitude")

# Upload "magnitude" variable to GPU

In [ ]:
neuron_layers

### Classifying a single image

In [ ]:
# Simulate
layer_spikes = [[] for i, _ in enumerate(neuron_layers)]
while model.timestep < PRESENT_TIMESTEPS:
    # Advance simulation
    model.step_time()

    # Loop through neuron layers
    for i, l in enumerate(neuron_layers):
        # Download spikes
        l.pull_current_spikes_from_device()        # Download spike emitted this timestep from GPU

        # Add to data structure
        layer_spikes[i].append(np.copy(l.current_spikes))        # Add spikes to data structure 

In [ ]:
# ----------------------------------------------------------------------------
# Plotting
# One plot for spikes in each neuron layer
# ----------------------------------------------------------------------------
import matplotlib.pyplot as plt

# Create a plot with axes for each
fig, axes = plt.subplots(len(neuron_layers), sharex=True)

# Loop through axes and their corresponding neuron populations
for ax, spk, lyr in zip(axes, layer_spikes, neuron_layers):
    spike_ids = np.concatenate(spk)
    spike_times = np.concatenate([np.ones_like(s) * i * TIMESTEP 
                                  for i, s in enumerate(spk)])

    # Plot spikes
    ax.scatter(spike_times, spike_ids, s=1)

    # Set title, axis labels
    ax.set_title(lyr.name)
    ax.set_ylabel("Spike number")
    ax.set_xlim((0, PRESENT_TIMESTEPS * TIMESTEP))
    ax.set_ylim((0, l.size))


# Add an x-axis label and translucent line showing the correct label
axes[-1].set_xlabel("Time [ms]")
axes[-1].hlines(testing_labels[0], xmin=0, xmax=PRESENT_TIMESTEPS,
                linestyle="--", color="gray", alpha=0.2)

# Show plot
plt.show()

### Classifying all images in the test set

In [ ]:
# Get views to efficiently access state variables
current_input_magnitude = current_input.vars["magnitude"].view
output_spike_count = neuron_layers[-1].vars["SpikeCount"].view
layer_voltages = [l.vars["V"].view for l in neuron_layers]       # Cache memory views of required state variables (?)

In [ ]:
# Simulate
num_correct = 0
while model.timestep < (PRESENT_TIMESTEPS * testing_images.shape[0]):
    # Calculate the timestep within the presentation
    timestep_in_example = model.timestep % PRESENT_TIMESTEPS           # Divide timestep into example index and timestep within example
    example = int(model.timestep // PRESENT_TIMESTEPS)

    # If this is the first timestep of presenting the example
    if timestep_in_example == 0:
        current_input_magnitude[:] = testing_images[example] * INPUT_CURRENT_SCALE       # Copy image into memory   view and upload to GPU
        current_input.push_var_to_device("magnitude")

        # Loop through all layers and their corresponding voltage views
        for l, v in zip(neuron_layers, layer_voltages):
            # Manually 'reset' voltage
            v[:] = 0.0                                                           # Set all neuron voltages to zero 

            # Upload
            l.push_var_to_device("V")                                            # Upload to GPU

        # Zero spike count
        output_spike_count[:] = 0
        neuron_layers[-1].push_var_to_device("SpikeCount")                       # Zero spike count for all output layer neurons and upload to GPU

    # Advance simulation
    model.step_time()                                                            # Advance simulation

    # If this is the LAST timestep of presenting the example
    if timestep_in_example == (PRESENT_TIMESTEPS - 1):
        # Download spike count from last layer
        neuron_layers[-1].pull_var_from_device("SpikeCount")                     # Download output layer spike count

        # Find which neuron spiked the most to get prediction
        predicted_label = np.argmax(output_spike_count)
        true_label = testing_labels[example]

        print("\tExample=%u, true label=%u, predicted label=%u" % (example,
                                                                   true_label,
                                                                   predicted_label))

        if predicted_label == true_label:
            num_correct += 1

print("Accuracy %f%%" % ((num_correct / float(testing_images.shape[0])) * 100.0))